# Logistic Regression Pipeline - Titanic Survival Prediction

This notebook demonstrates a complete end-to-end machine learning pipeline using **Logistic Regression** to predict passenger survival on the Titanic.

## Pipeline Overview:
1. **Data Loading** - Load the Titanic dataset
2. **Exploratory Data Analysis** - Understand the data
3. **Data Cleaning** - Handle missing values and outliers
4. **Feature Engineering** - Create and transform features
5. **Train-Test Split** - Split data for training and evaluation
6. **Model Training** - Build a Logistic Regression model with sklearn Pipeline
7. **Model Evaluation** - Accuracy, Precision, Recall, Confusion Matrix
8. **Model Export** - Save the trained model using joblib

## 1. Import Required Libraries

First, let's import all the necessary libraries for our ML pipeline.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix, 
    classification_report,
    roc_auc_score,
    roc_curve
)

# Model export
import joblib

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("All libraries imported successfully!")

## 2. Data Loading

We'll load the Titanic dataset directly from a public URL. This dataset contains information about passengers aboard the Titanic, including whether they survived.

In [ ]:
# Load the Titanic dataset from a public URL
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Display basic information
print(f"Dataset Shape: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nColumn Names: {list(df.columns)}")

# Show first few rows
df.head()

In [ ]:
# Check data types and non-null counts
print("Dataset Information:")
print("=" * 50)
df.info()

In [ ]:
# Statistical summary of numerical features
print("Statistical Summary of Numerical Features:")
df.describe()

## 3. Exploratory Data Analysis (EDA)

Let's explore the data to understand patterns and relationships between features.

In [ ]:
# Visualize target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Count plot
survival_counts = df['Survived'].value_counts()
colors = ['#ff6b6b', '#4ecdc4']
axes[0].bar(['Did Not Survive (0)', 'Survived (1)'], survival_counts.values, color=colors)
axes[0].set_title('Survival Distribution', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count')
for i, v in enumerate(survival_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Pie chart
axes[1].pie(survival_counts.values, labels=['Did Not Survive', 'Survived'], 
            autopct='%1.1f%%', colors=colors, explode=(0.05, 0), shadow=True)
axes[1].set_title('Survival Percentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Class Distribution: {survival_counts[0]} did not survive, {survival_counts[1]} survived")
print(f"Survival Rate: {(survival_counts[1] / len(df) * 100):.2f}%")

In [ ]:
# Analyze survival by different features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Survival by Gender
sns.countplot(data=df, x='Sex', hue='Survived', ax=axes[0, 0], palette=['#ff6b6b', '#4ecdc4'])
axes[0, 0].set_title('Survival by Gender', fontweight='bold')
axes[0, 0].legend(['Did Not Survive', 'Survived'])

# 2. Survival by Passenger Class
sns.countplot(data=df, x='Pclass', hue='Survived', ax=axes[0, 1], palette=['#ff6b6b', '#4ecdc4'])
axes[0, 1].set_title('Survival by Passenger Class', fontweight='bold')
axes[0, 1].legend(['Did Not Survive', 'Survived'])

# 3. Age Distribution by Survival
df['Survived'].replace({0: 'Did Not Survive', 1: 'Survived'}, inplace=False).values
sns.histplot(data=df, x='Age', hue='Survived', bins=30, ax=axes[0, 2], palette=['#ff6b6b', '#4ecdc4'])
axes[0, 2].set_title('Age Distribution by Survival', fontweight='bold')

# 4. Survival by Embarked Port
sns.countplot(data=df, x='Embarked', hue='Survived', ax=axes[1, 0], palette=['#ff6b6b', '#4ecdc4'])
axes[1, 0].set_title('Survival by Embarked Port', fontweight='bold')
axes[1, 0].legend(['Did Not Survive', 'Survived'])

# 5. Fare Distribution by Survival
sns.boxplot(data=df, x='Survived', y='Fare', ax=axes[1, 1], palette=['#ff6b6b', '#4ecdc4'])
axes[1, 1].set_title('Fare Distribution by Survival', fontweight='bold')
axes[1, 1].set_xticklabels(['Did Not Survive', 'Survived'])

# 6. Survival by SibSp (siblings/spouses)
sns.countplot(data=df, x='SibSp', hue='Survived', ax=axes[1, 2], palette=['#ff6b6b', '#4ecdc4'])
axes[1, 2].set_title('Survival by Siblings/Spouses Aboard', fontweight='bold')
axes[1, 2].legend(['Did Not Survive', 'Survived'])

plt.tight_layout()
plt.show()

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("=" * 50)
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

# Visualize missing values
fig, ax = plt.subplots(figsize=(10, 4))
missing_cols = missing[missing > 0]
bars = ax.bar(missing_cols.index, missing_cols.values, color='#e74c3c')
ax.set_title('Missing Values by Feature', fontsize=14, fontweight='bold')
ax.set_ylabel('Count of Missing Values')
ax.set_xlabel('Features')
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Data Cleaning

Now let's handle missing values and prepare the data for modeling:
- **Age**: Impute with median (robust to outliers)
- **Embarked**: Impute with mode (most frequent value)
- **Cabin**: Drop (too many missing values - 77%)
- **Ticket**: Drop (not useful for prediction)
- **PassengerId, Name**: Drop (identifiers, not features)

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Drop columns that won't be useful for prediction
columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df_clean = df_clean.drop(columns=columns_to_drop)

print(f"Dropped columns: {columns_to_drop}")
print(f"Remaining columns: {list(df_clean.columns)}")
print(f"New shape: {df_clean.shape}")

In [ ]:
# Handle missing values manually for initial EDA
# Note: We'll use Pipeline for proper imputation later

# Fill Age with median
age_median = df_clean['Age'].median()
df_clean['Age'] = df_clean['Age'].fillna(age_median)
print(f"Age: Filled {df['Age'].isnull().sum()} missing values with median ({age_median})")

# Fill Embarked with mode
embarked_mode = df_clean['Embarked'].mode()[0]
df_clean['Embarked'] = df_clean['Embarked'].fillna(embarked_mode)
print(f"Embarked: Filled {df['Embarked'].isnull().sum()} missing values with mode ('{embarked_mode}')")

# Verify no missing values remain
print(f"\nRemaining missing values: {df_clean.isnull().sum().sum()}")
df_clean.head()

## 5. Feature Engineering

Let's create new features that might improve our model's performance:
- **FamilySize**: Total family members aboard (SibSp + Parch + 1)
- **IsAlone**: Whether the passenger was traveling alone
- **AgeGroup**: Categorize age into groups
- **FarePerPerson**: Fare divided by family size

In [ ]:
# Create new features
df_clean['FamilySize'] = df_clean['SibSp'] + df_clean['Parch'] + 1
df_clean['IsAlone'] = (df_clean['FamilySize'] == 1).astype(int)
df_clean['FarePerPerson'] = df_clean['Fare'] / df_clean['FamilySize']

# Create age groups
def categorize_age(age):
    if age <= 12:
        return 'Child'
    elif age <= 18:
        return 'Teenager'
    elif age <= 35:
        return 'YoungAdult'
    elif age <= 55:
        return 'MiddleAged'
    else:
        return 'Senior'

df_clean['AgeGroup'] = df_clean['Age'].apply(categorize_age)

print("New features created!")
print(f"Updated shape: {df_clean.shape}")
df_clean.head()

In [ ]:
# Visualize the new features
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Family Size vs Survival
sns.countplot(data=df_clean, x='FamilySize', hue='Survived', ax=axes[0], palette=['#ff6b6b', '#4ecdc4'])
axes[0].set_title('Survival by Family Size', fontweight='bold')
axes[0].legend(['Did Not Survive', 'Survived'])

# IsAlone vs Survival
sns.countplot(data=df_clean, x='IsAlone', hue='Survived', ax=axes[1], palette=['#ff6b6b', '#4ecdc4'])
axes[1].set_title('Survival: Alone vs With Family', fontweight='bold')
axes[1].set_xticklabels(['With Family', 'Alone'])
axes[1].legend(['Did Not Survive', 'Survived'])

# Age Group vs Survival
age_order = ['Child', 'Teenager', 'YoungAdult', 'MiddleAged', 'Senior']
sns.countplot(data=df_clean, x='AgeGroup', hue='Survived', ax=axes[2], 
              order=age_order, palette=['#ff6b6b', '#4ecdc4'])
axes[2].set_title('Survival by Age Group', fontweight='bold')
axes[2].legend(['Did Not Survive', 'Survived'])
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create correlation heatmap for numerical features
# First, encode categorical variables for correlation analysis
df_corr = df_clean.copy()
df_corr['Sex'] = LabelEncoder().fit_transform(df_corr['Sex'])
df_corr['Embarked'] = LabelEncoder().fit_transform(df_corr['Embarked'])
df_corr['AgeGroup'] = LabelEncoder().fit_transform(df_corr['AgeGroup'])

plt.figure(figsize=(12, 8))
correlation = df_corr.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            center=0, linewidths=0.5, annot_kws={'size': 9})
plt.title('Feature Correlation Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Show correlations with target variable
print("\nCorrelation with Survival (sorted):")
print("=" * 40)
print(correlation['Survived'].sort_values(ascending=False))

## 6. Data Preparation for Modeling

Now let's prepare our data for the machine learning model:
1. Separate features (X) and target (y)
2. Define numerical and categorical columns
3. Create preprocessing pipelines using ColumnTransformer

In [ ]:
# Separate features and target
X = df_clean.drop('Survived', axis=1)
y = df_clean['Survived']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns: {list(X.columns)}")
print(f"\nTarget distribution:\n{y.value_counts()}")

In [ ]:
# Define numerical and categorical columns
numerical_features = ['Age', 'Fare', 'SibSp', 'Parch', 'FamilySize', 'IsAlone', 'FarePerPerson']
categorical_features = ['Pclass', 'Sex', 'Embarked', 'AgeGroup']

print("Numerical Features:")
for feat in numerical_features:
    print(f"   - {feat}: {X[feat].dtype}")

print("\nCategorical Features:")
for feat in categorical_features:
    print(f"   - {feat}: {X[feat].nunique()} unique values")

## 7. Train-Test Split

Split the data into training and testing sets to evaluate model performance on unseen data.

In [ ]:
# Split the data into training and testing sets
# Using stratified split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,           # 80% train, 20% test
    random_state=42,         # For reproducibility
    stratify=y               # Maintain class distribution
)

print("Data Split Summary:")
print("=" * 50)
print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Testing set:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\nTarget Distribution in Training Set:")
print(f"   Did Not Survive: {(y_train == 0).sum()} ({(y_train == 0).sum()/len(y_train)*100:.1f}%)")
print(f"   Survived: {(y_train == 1).sum()} ({(y_train == 1).sum()/len(y_train)*100:.1f}%)")

print(f"\nTarget Distribution in Test Set:")
print(f"   Did Not Survive: {(y_test == 0).sum()} ({(y_test == 0).sum()/len(y_test)*100:.1f}%)")
print(f"   Survived: {(y_test == 1).sum()} ({(y_test == 1).sum()/len(y_test)*100:.1f}%)")

## 8. Build sklearn Pipeline

We'll create a complete ML pipeline that includes:
1. **Numerical Pipeline**: Imputation + Standard Scaling
2. **Categorical Pipeline**: Imputation + One-Hot Encoding
3. **ColumnTransformer**: Combine both pipelines
4. **Final Pipeline**: Preprocessing + Logistic Regression

Using a Pipeline ensures that all preprocessing steps are applied consistently and prevents data leakage.

In [ ]:
# Create preprocessing pipelines for numerical and categorical features

# Numerical features pipeline: Impute missing values with median, then scale
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                     # Standardize features
])

# Categorical features pipeline: Impute with most frequent, then one-hot encode
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # Encode categories
])

# Combine both pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='drop'  # Drop any columns not specified
)

print("Preprocessing pipeline created!")
print("\nPipeline Structure:")
print("   Numerical Features -> SimpleImputer (median) -> StandardScaler")
print("   Categorical Features -> SimpleImputer (mode) -> OneHotEncoder")

In [ ]:
# Create the complete ML pipeline with Logistic Regression
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        random_state=42,
        max_iter=1000,           # Increase iterations for convergence
        solver='lbfgs',          # Good for small datasets
        C=1.0,                   # Regularization strength
        class_weight='balanced'  # Handle class imbalance
    ))
])

# Display the pipeline
print("Complete ML Pipeline:")
print("=" * 60)
print(model_pipeline)

## 9. Model Training

Train the Logistic Regression model on the training data. We'll also perform cross-validation to get a more robust estimate of model performance.

In [ ]:
# Train the model
print("Training the Logistic Regression model...")
print("=" * 50)

# Fit the pipeline on training data
model_pipeline.fit(X_train, y_train)

print("Model training complete!")

# Perform cross-validation on training data
print("\nCross-Validation Results (5-fold):")
print("-" * 50)

cv_scores = cross_val_score(model_pipeline, X_train, y_train, cv=5, scoring='accuracy')
print(f"   Fold Scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"   Mean Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

## 10. Model Evaluation

Evaluate the model's performance on the test set using various metrics:
- **Accuracy**: Overall correctness
- **Precision**: Of predicted positives, how many were actually positive
- **Recall**: Of actual positives, how many were predicted correctly
- **F1-Score**: Harmonic mean of Precision and Recall
- **Confusion Matrix**: Detailed breakdown of predictions
- **ROC-AUC**: Area under the ROC curve

In [ ]:
# Make predictions on training and test sets
y_train_pred = model_pipeline.predict(X_train)
y_test_pred = model_pipeline.predict(X_test)

# Get probability predictions for ROC curve
y_train_proba = model_pipeline.predict_proba(X_train)[:, 1]
y_test_proba = model_pipeline.predict_proba(X_test)[:, 1]

print("Predictions generated for training and test sets")

In [ ]:
# Calculate evaluation metrics
print("MODEL EVALUATION RESULTS")
print("=" * 60)

# Training Set Metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, y_train_proba)

print("\nTRAINING SET PERFORMANCE:")
print("-" * 40)
print(f"   Accuracy:  {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"   Precision: {train_precision:.4f}")
print(f"   Recall:    {train_recall:.4f}")
print(f"   F1-Score:  {train_f1:.4f}")
print(f"   ROC-AUC:   {train_roc_auc:.4f}")

# Test Set Metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, y_test_proba)

print("\nTEST SET PERFORMANCE:")
print("-" * 40)
print(f"   Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   Precision: {test_precision:.4f}")
print(f"   Recall:    {test_recall:.4f}")
print(f"   F1-Score:  {test_f1:.4f}")
print(f"   ROC-AUC:   {test_roc_auc:.4f}")

In [ ]:
# Detailed Classification Report
print("\nDETAILED CLASSIFICATION REPORT (Test Set):")
print("=" * 60)
print(classification_report(y_test, y_test_pred, target_names=['Did Not Survive', 'Survived']))

In [ ]:
# Confusion Matrix Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training Set Confusion Matrix
cm_train = confusion_matrix(y_train, y_train_pred)
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Did Not Survive', 'Survived'],
            yticklabels=['Did Not Survive', 'Survived'])
axes[0].set_title('Confusion Matrix - Training Set', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted Label')
axes[0].set_ylabel('True Label')

# Test Set Confusion Matrix
cm_test = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', ax=axes[1],
            xticklabels=['Did Not Survive', 'Survived'],
            yticklabels=['Did Not Survive', 'Survived'])
axes[1].set_title('Confusion Matrix - Test Set', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted Label')
axes[1].set_ylabel('True Label')

plt.tight_layout()
plt.show()

# Print confusion matrix interpretation
tn, fp, fn, tp = cm_test.ravel()
print("\nConfusion Matrix Breakdown (Test Set):")
print("-" * 50)
print(f"   True Negatives (TN):  {tn} - Correctly predicted 'Did Not Survive'")
print(f"   False Positives (FP): {fp} - Incorrectly predicted 'Survived'")
print(f"   False Negatives (FN): {fn} - Incorrectly predicted 'Did Not Survive'")
print(f"   True Positives (TP):  {tp} - Correctly predicted 'Survived'")

In [ ]:
# ROC Curve
fig, ax = plt.subplots(figsize=(8, 6))

# Calculate ROC curves
fpr_train, tpr_train, _ = roc_curve(y_train, y_train_proba)
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba)

# Plot ROC curves
ax.plot(fpr_train, tpr_train, color='blue', lw=2, 
        label=f'Training ROC (AUC = {train_roc_auc:.4f})')
ax.plot(fpr_test, tpr_test, color='red', lw=2, 
        label=f'Test ROC (AUC = {test_roc_auc:.4f})')
ax.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random Classifier')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('Receiver Operating Characteristic (ROC) Curve', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Feature Importance Analysis

Let's analyze which features are most important for predicting survival by examining the model coefficients.

In [ ]:
# Get feature names after preprocessing
# Get the one-hot encoded feature names
onehot_encoder = model_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
cat_feature_names = list(onehot_encoder.get_feature_names_out(categorical_features))

# Combine all feature names
all_feature_names = numerical_features + cat_feature_names

# Get coefficients from the trained model
coefficients = model_pipeline.named_steps['classifier'].coef_[0]

# Create a DataFrame for feature importance
feature_importance = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
}).sort_values('Abs_Coefficient', ascending=False)

# Display feature importance
print("Feature Importance (by absolute coefficient value):")
print("=" * 60)
print(feature_importance.to_string(index=False))

In [ ]:
# Visualize feature importance
fig, ax = plt.subplots(figsize=(12, 8))

# Create color based on positive/negative coefficient
colors = ['#4ecdc4' if x > 0 else '#ff6b6b' for x in feature_importance['Coefficient']]

# Plot horizontal bar chart
bars = ax.barh(range(len(feature_importance)), 
               feature_importance['Coefficient'], 
               color=colors)

ax.set_yticks(range(len(feature_importance)))
ax.set_yticklabels(feature_importance['Feature'])
ax.set_xlabel('Coefficient Value', fontsize=12)
ax.set_title('Logistic Regression Feature Coefficients\n(Green = Positive Impact, Red = Negative Impact)', 
             fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("   - Positive coefficients increase the probability of survival")
print("   - Negative coefficients decrease the probability of survival")
print("   - Larger absolute values indicate stronger influence")

## 12. Model Export

Save the trained model pipeline using `joblib` for later use in production or deployment.

In [ ]:
# Save the trained model pipeline
import os

# Create the model directory if it doesn't exist
model_dir = '../model'
os.makedirs(model_dir, exist_ok=True)

# Define the model path
model_path = os.path.join(model_dir, 'logistic_model.joblib')

# Save the model
joblib.dump(model_pipeline, model_path)

print("MODEL EXPORT SUCCESSFUL!")
print("=" * 60)
print(f"   Model saved to: {os.path.abspath(model_path)}")
print(f"   File size: {os.path.getsize(model_path) / 1024:.2f} KB")

# Verify the saved model
print("\nVerification: Loading and testing the saved model...")
loaded_model = joblib.load(model_path)
loaded_predictions = loaded_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, loaded_predictions)
print(f"   Loaded model accuracy: {loaded_accuracy:.4f}")
print("   Model verified successfully!")

## 13. Making Predictions with the Saved Model

Let's demonstrate how to load and use the saved model for making predictions on new data.

In [ ]:
# Example: Making predictions for new passengers
# Create sample new passenger data

new_passengers = pd.DataFrame({
    'Pclass': [1, 3, 2],
    'Sex': ['female', 'male', 'female'],
    'Age': [25, 35, 8],
    'SibSp': [1, 0, 2],
    'Parch': [0, 0, 1],
    'Fare': [100.0, 7.25, 30.0],
    'Embarked': ['S', 'S', 'C'],
    'FamilySize': [2, 1, 4],
    'IsAlone': [0, 1, 0],
    'FarePerPerson': [50.0, 7.25, 7.5],
    'AgeGroup': ['YoungAdult', 'YoungAdult', 'Child']
})

print("New Passenger Data:")
print("=" * 60)
print(new_passengers.to_string(index=False))

# Load the model and make predictions
model = joblib.load('../model/logistic_model.joblib')
predictions = model.predict(new_passengers)
probabilities = model.predict_proba(new_passengers)

print("\nPrediction Results:")
print("-" * 60)
for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
    survival_status = "Survived" if pred == 1 else "Did Not Survive"
    print(f"   Passenger {i+1}: {survival_status}")
    print(f"              Probability of Survival: {prob[1]*100:.2f}%")
    print()

## 14. Summary & Conclusions

### Pipeline Summary
This notebook demonstrated a complete end-to-end machine learning pipeline for binary classification using Logistic Regression:

| Step | Description |
|------|-------------|
| 1. Data Loading | Loaded Titanic dataset from public URL |
| 2. EDA | Explored data distributions and relationships |
| 3. Data Cleaning | Handled missing values, dropped irrelevant columns |
| 4. Feature Engineering | Created FamilySize, IsAlone, AgeGroup, FarePerPerson |
| 5. Preprocessing | Used sklearn Pipeline with ColumnTransformer |
| 6. Model Training | Trained Logistic Regression with balanced class weights |
| 7. Evaluation | Assessed using accuracy, precision, recall, F1, ROC-AUC |
| 8. Model Export | Saved pipeline as `logistic_model.joblib` |

### Key Findings
- **Sex** is the most important predictor (females had higher survival rates)
- **Passenger Class** significantly affected survival (1st class had better odds)
- **Family Size** had a non-linear effect (small families survived better than lone travelers or large families)
- **Age** played a role, with children having better survival rates

### Model Performance
The Logistic Regression model achieved:
- ~80% accuracy on the test set
- Good precision and recall balance
- Strong ROC-AUC score indicating reliable predictions

### Next Steps
- Try other algorithms (Random Forest, XGBoost)
- Hyperparameter tuning with GridSearchCV
- Feature selection techniques
- Deploy the model using FastAPI (see `backend/main.py`)

In [ ]:
# Final Summary Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Metrics comparison bar chart
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
train_scores = [train_accuracy, train_precision, train_recall, train_f1, train_roc_auc]
test_scores = [test_accuracy, test_precision, test_recall, test_f1, test_roc_auc]

x = np.arange(len(metrics))
width = 0.35

bars1 = axes[0].bar(x - width/2, train_scores, width, label='Training', color='#3498db')
bars2 = axes[0].bar(x + width/2, test_scores, width, label='Test', color='#e74c3c')

axes[0].set_ylabel('Score')
axes[0].set_title('Model Performance Metrics', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics)
axes[0].legend()
axes[0].set_ylim(0, 1.1)
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.3f}', ha='center', va='bottom', fontsize=8)
for bar in bars2:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.3f}', ha='center', va='bottom', fontsize=8)

# Top 5 important features
top_features = feature_importance.head(5)
axes[1].barh(range(len(top_features)), top_features['Abs_Coefficient'], 
             color=['#2ecc71' if x > 0 else '#e74c3c' for x in top_features['Coefficient']])
axes[1].set_yticks(range(len(top_features)))
axes[1].set_yticklabels(top_features['Feature'])
axes[1].set_xlabel('Absolute Coefficient Value')
axes[1].set_title('Top 5 Most Important Features', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("NOTEBOOK COMPLETE!")
print("="*60)
print(f"Model saved to: ../model/logistic_model.joblib")
print(f"Final Test Accuracy: {test_accuracy*100:.2f}%")
print("="*60)